In [ ]:
# Dependencies
import pandas as pd
from pathlib import Path

In [ ]:
# Load in file and store filepath in a variable
transit_file = Path("Resource/Metro-Systems-Worldwide.csv")

In [ ]:
# Read and display the CSV with Pandas
transit_df = pd.read_csv(transit_file)
transit_df.head()

In [ ]:
# List all the columns in the table
transit_df.columns

In [ ]:
# Use .rename(columns={}) to rename columns
transit_renamed_df = transit_df.rename(columns={"Country\r\nregion":"Country region", 
                                                "Service\r\nopened":"Service opened",
                                                "Last\r\nexpanded":"Last expanded",
                                                "Annual ridership\r\n(millions)":"Annual ridership (millions)"})
transit_renamed_df.head()

In [ ]:
transit_renamed_df.count()

In [ ]:
drop_na_df = transit_renamed_df.dropna(how='any')

In [ ]:
transit_renamed_df.count()

In [ ]:
transit_renamed_df.count()

In [ ]:
# TRY to clean up Last expanded and Annual ridership (millions) category. 
transit_renamed_df['Last expanded'] = transit_renamed_df['Last expanded'].replace({'–': 'NA'})
transit_renamed_df.head()

transit_renamed_df['Annual ridership (millions)'] = transit_renamed_df['Annual ridership (millions)'].replace({'n/a': 'Hello'})
transit_renamed_df.head(15)